In [16]:
import json
import os
import ast
import glob
import pandas as pd
import numpy as np

In [59]:
import gensim.corpora as corpora
import gensim
from gensim.test.utils import datapath

In [29]:
import sys
sys.path.append('..')
from src.features.segment_transcript import CreateBlocksteBlocks

In [8]:
groundbase_dir = '../data/raw/groundbase'
transcripts_dir = os.path.join(groundbase_dir,'transcripts')
topic_dataset_path = os.path.join(groundbase_dir,'dataset.csv')
transcript_filespath = glob.glob(groundbase_dir + '/transcripts/*.json')

'''Read the transcript'''
transcripts_jsons = {}
for fl in transcript_filespath:
    with open(fl,encoding="utf8") as f:
        transcript =ast.literal_eval(f.read()) #json.load(f)
        vid = fl.split('\\')[-1].split('.')[0]
        #print(vid)
        transcripts_jsons[vid] = transcript

In [11]:
df_videos = pd.read_csv(topic_dataset_path)
df_videos.head()

,shift index,topic shifts(ends),label,video,subject,youtube link,length,video id
0,0,00:11:17,"Syllabus , Textbook and references",Mod-01 Lec-01 Foundation of Scientific Computi...,Foundation of Scientific Computing,https://www.youtube.com/watch?v=MkiUBJcgdUY,01:05:06,MkiUBJcgdUY
1,1,00:12:45,course grading,Mod-01 Lec-01 Foundation of Scientific Computi...,Foundation of Scientific Computing,https://www.youtube.com/watch?v=MkiUBJcgdUY,01:05:06,MkiUBJcgdUY
2,2,00:20:57,relationship between computing and science,Mod-01 Lec-01 Foundation of Scientific Computi...,Foundation of Scientific Computing,https://www.youtube.com/watch?v=MkiUBJcgdUY,01:05:06,MkiUBJcgdUY
3,3,00:22:37,implemntation,Mod-01 Lec-01 Foundation of Scientific Computi...,Foundation of Scientific Computing,https://www.youtube.com/watch?v=MkiUBJcgdUY,01:05:06,MkiUBJcgdUY
4,4,00:24:45,scientific computing uses,Mod-01 Lec-01 Foundation of Scientific Computi...,Foundation of Scientific Computing,https://www.youtube.com/watch?v=MkiUBJcgdUY,01:05:06,MkiUBJcgdUY


In [57]:
'''Find the number of topics in each video'''
n_video_topic = df_videos.groupby(['video id'],as_index=False)['shift index'].max().sum().values.tolist()[1]

In [25]:
'''Set the training set as a whole text to train the lda'''
text = np.concatenate([[brth for brth in transcripts_jsons[trs]]
                       for trs in transcripts_jsons.keys()])

In [27]:
text.shape
text[0]

{'text': 'hello my name is Tommy and welcome to',
 'start': 0.35,
 'duration': 5.02}

In [48]:
block_handler =  CreateBlocks(text)
blocks = block_handler.partion_by_sliding_windows(20,20)

In [47]:
#blocks = np.array(blocks).reshape(-1,20)

ValueError: cannot reshape array of size 1250 into shape (20)

In [49]:
id2word = corpora.Dictionary(blocks)
corpus = [id2word.doc2bow(doc) for doc in blocks]

In [58]:
'''You need to train the LDA model you need to make here some loops and stabilizing params'''
# low alpha means each document is only represented by a small number of topics, and vice versa
# low eta means each topic is only represented by a small number of words, and vice versa
# https://www.kaggle.com/ktattan/lda-and-document-similarity

lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                  num_topics=n_video_topic,
                                                  id2word=id2word#,
                                                  #alpha=vectorizing_params['alpha']#, # consider this to be auto
                                                  #eta=vectorizing_params['eta'],
                                                  #chunksize=vectorizing_params['chunksize'],
                                                  #minimum_probability=vectorizing_params['minimum_probability'],
                                                  #passes=10
                                                  )

In [66]:
my_path = '../models/LDA MODELS/lda_model'
lda.save(my_path)

In [67]:
#lda = gensim.models.ldamodel.LdaModel.load(my_path)

In [68]:
'''

A problem a rised when we try to classify new text without a known words to the model
since it's a bag of words, It's can be problematic.
And the prediction of new data is getting the distribution of topics in a document
One way to solve it is to run it on a lot of external corpus but this is for a another resreach

'''